# MVP-V: 전체 파이프라인 통합

이 노트북은 MVP-V의 전체 파이프라인을 통합합니다: 인지 → 깊이 → 추적 → 계획

## 목표
- 전체 파이프라인 통합: 인지 → 깊이 → 추적
- 규칙 기반 계획 (안전거리 유지, 차선 중앙 유지)
- 실시간 결과 시각화

In [ ]:
# Cell 1: 라이브러리 및 모듈 임포트
import numpy as np
import cv2
import matplotlib.pyplot as plt
from pathlib import Path
from ultralytics import YOLO

# 이전 노트북에서 정의한 함수들 임포트
# (실제로는 별도 모듈로 분리하거나 코드를 복사해야 함)
import sys
sys.path.append(str(Path.cwd().parent.parent))

print("모듈 임포트 완료")

In [ ]:
# Cell 2: Vision Pipeline 클래스 정의
class VisionPipeline:
    """
    비전 기반 자율주행 파이프라인
    Perception → Depth → Tracking → Planning
    """
    def __init__(self, yolo_model_path='yolov8n.pt', calib=None):
        self.yolo_model = YOLO(yolo_model_path)
        self.calib = calib
        self.tracker = None  # 추적기는 별도 초기화 필요
        
    def process_frame(self, image_path):
        """
        단일 프레임 처리
        """
        # 1. Perception: YOLO 객체 탐지
        results = self.yolo_model(str(image_path))
        detections = results[0]
        
        # 2. Depth: 스테레오 깊이 추정 (여기서는 생략, 별도 노트북 참조)
        # depth_map = compute_depth_map(left_img, right_img, ...)
        
        # 3. Tracking: 객체 추적
        # tracks = track_objects(self.tracker, detections, frame_idx)
        
        # 4. Planning: 규칙 기반 계획
        # plan = self.plan(tracks, depth_map)
        
        return {
            'detections': detections,
            # 'depth_map': depth_map,
            # 'tracks': tracks,
            # 'plan': plan
        }
    
    def plan(self, tracks, depth_map, safe_distance=50.0):
        """
        규칙 기반 경로 계획
        - 전방 안전거리 유지 (ACC 느낌)
        - 차선 중앙 유지
        """
        plan = {
            'action': 'keep_lane',
            'speed': 50.0,  # km/h
            'safe_distance': safe_distance,
            'warnings': []
        }
        
        # 전방 객체 확인
        for track in tracks:
            # 깊이 정보를 사용하여 거리 확인
            # distance = get_distance(track, depth_map)
            # if distance < safe_distance:
            #     plan['action'] = 'decelerate'
            #     plan['warnings'].append(f"전방 객체 감지: {distance:.1f}m")
        
        return plan

print("Vision Pipeline 클래스 정의 완료")

In [ ]:
# Cell 3: 통합 파이프라인 실행
def run_pipeline(pipeline, image_dir, num_frames=5):
    """
    여러 프레임에 대해 파이프라인 실행
    """
    image_files = sorted(image_dir.glob("*.png"))[:num_frames]
    results = []
    
    for frame_idx, image_path in enumerate(image_files):
        print(f"프로세싱 Frame {frame_idx+1}/{len(image_files)}: {image_path.name}")
        result = pipeline.process_frame(image_path)
        results.append((image_path, result))
    
    return results

# 테스트
project_root = Path.cwd().parent.parent
dataset_root = project_root / "dataset" / "kitti"
train_images_dir = dataset_root / "images" / "train"

if train_images_dir.exists():
    pipeline = VisionPipeline(yolo_model_path='yolov8n.pt')
    # results = run_pipeline(pipeline, train_images_dir, num_frames=5)
    print("파이프라인 준비 완료")
else:
    print("이미지 디렉토리를 찾을 수 없습니다.")

In [ ]:
# Cell 4: 결과 시각화
def visualize_pipeline_result(image_path, result):
    """
    파이프라인 결과 통합 시각화
    """
    image = cv2.imread(str(image_path))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    fig, axes = plt.subplots(1, 2, figsize=(20, 8))
    
    # 원본 이미지
    axes[0].imshow(image)
    axes[0].set_title("원본 이미지", fontsize=14)
    axes[0].axis('off')
    
    # 탐지 결과
    annotated = result['detections'].plot()
    axes[1].imshow(annotated)
    axes[1].set_title("파이프라인 결과 (탐지)", fontsize=14)
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()
    return fig

print("시각화 함수 정의 완료")